# 3rd Level Model Structure: Everything (Linking Stages)

In [ ]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

In [ ]:
from pyomo import environ as pe
from modules.global_set import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF

model = pe.ConcreteModel()

# Global Set

In [ ]:
model.TRAY = pe.RangeSet(1,1)

# Construct Reactive Stages

In [ ]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

# Construct a single condenser

In [ ]:
model.condenser = pe.Block(rule=condenser_stage_rule)

# Linking Stage Variables

### Vapor Between Reactive Stages

In [ ]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [ ]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [ ]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [ ]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

# Fixing Redundent Stream Variables

In [ ]:
# condenser
model.condenser.VLE_block.n_ave.fix(58)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [ ]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

In [ ]:
check_DOF(pe,model)

# Load Operating Parameters

In [ ]:
# condenser
model.condenser.P.fix(19)
#model.condenser.T.fix(30+273.15)
model.condenser.T.setub(30+273.15)
model.condenser.L['out'].setub(6)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(2000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(58)
    
    model.reactive[j].F.fix(2)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(0.4)
    model.reactive[j].z['H2'].fix(0.6)
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    model.reactive[j].Q_main.fix(0)
    # model.reactive[j].T.setlb(270+273.15)

In [ ]:
check_DOF(pe,model)

In [ ]:
model.obj = pe.Objective(expr = model.reactive[1].L['out'] ,sense=pe.maximize)

In [ ]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma97'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [ ]:
add_dual(pe,model)

In [ ]:
# with open('../saved_solutions/1_stage_condenser.pickle', 'rb') as f:
#         results = pickle.load(f)

In [ ]:
# opt.options['warm_start_init_point'] = 'yes'
# opt.options['warm_start_bound_push'] = 1e-20
# opt.options['warm_start_mult_bound_push'] = 1e-20
# opt.options['mu_init'] = 1e-6

In [ ]:
# model.solutions.load_from(results)

In [ ]:
# model.ipopt_zL_in.update(model.ipopt_zL_out)
# model.ipopt_zU_in.update(model.ipopt_zU_out)

In [ ]:
results = opt.solve(model,tee=True)

In [ ]:
model.ipopt_zL_in.update(model.ipopt_zL_out)
model.ipopt_zU_in.update(model.ipopt_zU_out)

In [ ]:
beautify2(pe,model)

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/1_stage_condenser_n58_tmp.pickle','wb') as f:
#     pickle.dump(results,f)

### As you can see, because Q is left unfixed, certain energy constraints has 0 multiplier, not active

In [ ]:
for i in model.dual:
    if abs(model.dual[i]) == 0:
        print('{:40s}'.format(str(i)),'\t\t',model.dual[i])

# Iterative Data Analysis

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
opt.options['linear_solver'] = 'ma86'

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.condenser.L['P'] ,sense=pe.maximize)
model.condenser.T.fixed = True
model.condenser.L['out'].fixed=True
for j in model.reactive:
    model.reactive[j].Q_main.fixed=False
    model.reactive[j].T.fixed=True

In [ ]:
results = opt.solve(model,tee=True)

In [ ]:
update_dual(pe,model)

In [ ]:
Trange = np.linspace(model.reactive[1].T.value,200+273.15,50)

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; rf_data['Re'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []

for i in model.reactive[1].r_total_comp:
    rf_data['r'][i] = []

for t in Trange:
    model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved, Solution T = {} K'.format(model.reactive[1].T.value))
    rf_data['T'].append(model.reactive[1].T.value)
    rf_data['Q'].append(model.reactive[1].Q_main.value)
    rf_data['V'].append(model.condenser.V['out'].value)
    rf_data['L'].append(model.condenser.L['P'].value)
    rf_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        rf_data['r'][i].append(model.reactive[1].r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.reactive[1].y['H2O'].value)
    rf_data['y_CO'].append(model.reactive[1].y['CO'].value)
    rf_data['y_H2'].append(model.reactive[1].y['H2'].value)
    rf_data['y_CO2'].append(model.reactive[1].y['CO2'].value)
    rf_data['r_WGS'].append(model.reactive[1].kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.reactive[1].kinetics_block.r_FT_total.value)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
line1 = ax1.plot(rf_data['T'],rf_data['Q'], 'b-')
ax1.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax1.set_ylabel('Q (MW)', color='b',fontsize=18)
ax1.tick_params('y', colors='b',labelsize=18)
ax1.tick_params('x', colors='k',labelsize=18)
ax1.grid()

ax2 = ax1.twinx()
line2 = ax2.plot(rf_data['T'],rf_data['r_FT'], 'r')
line3 = ax2.plot(rf_data['T'],rf_data['r_WGS'], 'r--')
ax2.set_ylabel('$r_{FT}$: (kmol/s Carbon Number)', color='r',fontsize=18)
ax2.tick_params('y', colors='r',labelsize=18)

ax1.legend(line1+line2+line3,['Q','$r_{FT}$','$r_{WGS}$'],fontsize=18,loc=5)
ax1.set_title('Heat Removal Rate and Reaction Rate vs T',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['y_CO'])
ax.plot(rf_data['T'],rf_data['y_H2'])
ax.plot(rf_data['T'],rf_data['y_CO2'])
ax.plot(rf_data['T'],rf_data['y_H2O'])

ax.legend(['$y_{CO}$','$y_{H2}$','$y_{CO2}$','$y_{H2O}$'],fontsize=18)
ax.set_title('Gas Phase Inorganic Composition vs T',fontsize=18)

ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['r']['CO'])
ax.plot(rf_data['T'],rf_data['r']['H2'])
ax.plot(rf_data['T'],rf_data['r']['CO2'])
ax.plot(rf_data['T'],rf_data['r']['H2O'])

ax.legend(['$r_{CO}$','$r_{H2}$','$r_{CO2}$','$r_{H2O}$'],fontsize=18)
ax.set_title('Inorganic Reaction Rate vs T',fontsize=18)

ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
reaction_data = trans_product_mole(rf_data['r'])['unscaled']
reaction_data_scaled = trans_product_mole(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data_scaled['c1'], reaction_data_scaled['c2'],reaction_data_scaled['c3'],reaction_data_scaled['c4'],\
            reaction_data_scaled['napha'],reaction_data_scaled['gasoline'],reaction_data_scaled['diesel'],reaction_data_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mole) vs T',fontsize=18)
ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data['c1'], reaction_data['c2'],reaction_data['c3'],reaction_data['c4'],\
            reaction_data['napha'],reaction_data['gasoline'],reaction_data['diesel'],reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (Mole) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
mass_data = {}
for i in m.COMP_ORG:
    mass_data[i] = np.array(rf_data['r'][i])*u.cal_MW(i)

In [ ]:
mass_reaction_data = u.trans_product(mass_data)
mass_reaction_scaled = u.trans_product_scaled(mass_data)

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_scaled['c1'], mass_reaction_scaled['c2'],mass_reaction_scaled['c3'],mass_reaction_scaled['c4'],\
            mass_reaction_scaled['napha'],mass_reaction_scaled['gasoline'],mass_reaction_scaled['diesel'],mass_reaction_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mass) vs T',fontsize=18)
ax.set_ylabel('Mass Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_data['c1'], mass_reaction_data['c2'],mass_reaction_data['c3'],mass_reaction_data['c4'],\
            mass_reaction_data['napha'],mass_reaction_data['gasoline'],mass_reaction_data['diesel'],mass_reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (mass) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kg/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()